In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.basketball-reference.com/boxscores/201606190GSW.html'
page = requests.get(url)

In [3]:
soup = BeautifulSoup(page.content)

In [4]:
tables = [x for x in soup.find_all('table') if x.get('id') in ['box-CLE-game-basic', 'box-GSW-game-basic']]

In [40]:
def mp_str_to_float(x) :
  minutes, seconds = map(int, x.split(':'))
  return minutes + seconds / 60

dfs = []
for table in tables :
    
  tm = table.get('id').split('-')[1]
  df = pd.read_html(str(table))[0][:-1].drop(5)
  df.columns = df.columns.droplevel(0)
  df['href'] = [x['href'].split('/')[-1].split('.')[0] for x in table.find_all('a', href = True)]
  df['FG'] = pd.to_numeric(df['FG'], errors = 'coerce')
  df = df[df['FG'].notna()]
  df['MP'] = df['MP'].apply(mp_str_to_float)
  df = df[['Starters', 'href', 'MP']].rename(columns = {'Starters': 'Player'})
  df['Tm'] = tm
  dfs.append(df)

players = pd.concat(dfs).reset_index(drop = True)

In [46]:
tm_col  = {'CLE': 'black', 'GSW': 'white'}
tm_name = {'CLE': 'cavaliers', 'GSW': 'warriors'}

In [48]:
for _, player in players.iterrows() :
    prompt = player['Player'] + ' ' + tm_name.get(player['Tm']) + ' 2016 ' + tm_col.get(player['Tm'])

    print(player['Player'], ':', prompt)

LeBron James : LeBron James cavaliers 2016 black
Kyrie Irving : Kyrie Irving cavaliers 2016 black
J.R. Smith : J.R. Smith cavaliers 2016 black
Tristan Thompson : Tristan Thompson cavaliers 2016 black
Kevin Love : Kevin Love cavaliers 2016 black
Richard Jefferson : Richard Jefferson cavaliers 2016 black
Iman Shumpert : Iman Shumpert cavaliers 2016 black
Mo Williams : Mo Williams cavaliers 2016 black
Draymond Green : Draymond Green warriors 2016 white
Klay Thompson : Klay Thompson warriors 2016 white
Stephen Curry : Stephen Curry warriors 2016 white
Harrison Barnes : Harrison Barnes warriors 2016 white
Festus Ezeli : Festus Ezeli warriors 2016 white
Andre Iguodala : Andre Iguodala warriors 2016 white
Shaun Livingston : Shaun Livingston warriors 2016 white
Anderson Varejão : Anderson Varejão warriors 2016 white
Marreese Speights : Marreese Speights warriors 2016 white
Leandro Barbosa : Leandro Barbosa warriors 2016 white


In [43]:
player['Player']

'Leandro Barbosa'